In [172]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snb
import warnings
warnings.filterwarnings("ignore")
import os
import sys
from datetime import datetime

In [173]:
import tensorflow as tf

In [174]:
from tensorflow import keras

In [175]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping , TensorBoard
from keras.saving import save_model

In [176]:
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy

In [177]:
def get_recent_log_dir():
    log_path = os.path.join(os.getcwd(), "log", "fit")
    log_files = sorted(os.listdir(log_path))
    return os.path.join(log_path, log_files[-1])

def launch_tensorboard():
    ipython = get_ipython()

    # Load or reload the extension
    try:
        ipython.run_line_magic('reload_ext', 'tensorboard')
    except:
        ipython.run_line_magic('load_ext', 'tensorboard')

    # Get latest log directory
    log_dir = get_recent_log_dir()
    print("Launching TensorBoard at:", log_dir)

    # Start TensorBoard
    ipython.run_line_magic('tensorboard', f'--logdir "{log_dir}"')


In [178]:
def get_tensorboard_callback():
    log_path=datetime.now().strftime("%d_%m_%y__%H_%M_%S")
    log_path= "log/fit/" + log_path 

    return TensorBoard(
            log_dir=log_path,
            histogram_freq=1
        )

In [179]:
early_stopping_callback=EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

In [180]:
def sequential_model(x_train,y_train):
  model=Sequential()
  model.add(
      Dense(
          units=64,
          activation='relu',
          input_shape=(x_train.shape[1],)
      )
  )
  model.add(
      Dense(
          units=32,
          activation="relu"
      )
  )
  model.add(
      Dense(
          units=1,
          activation="tanh"
      )
  )

In [181]:
def eval_model(model_input_shape):
  n=model_input_shape
  model=Sequential()
  model.add(
      Dense(
          units=2*n,
          activation='relu',
          input_shape=(n,)
      )
  )
  model.add(
      Dense(
          units=4*n,
          activation="relu"
      )
  )
  model.add(
      Dense(
          units=2*n,
          activation="relu"
      )
  )
  model.add(
      Dense(
          units=1,
          activation="tanh"
      )
  )

  print(model.summary())

  optimizer=Adam(learning_rate=0.005)
  loss=BinaryCrossentropy()
  metrics=BinaryAccuracy()

  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=[metrics]
  )
  return model

In [182]:
model=eval_model(12)

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (None, 24)             │           312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 48)             │         1,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 24)             │         1,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,713 (10.60 KB)

 Trainable params: 2,713 (10.60 KB)

 Non-trainable params: 0 (0.00 B)

None


In [183]:
train_data=pd.read_csv("train_data.csv")
test_data=pd.read_csv("test_data.csv")

In [184]:
train_data.columns

Index(['Unnamed: 0', 'standard_scaler__CreditScore', 'standard_scaler__Age',
       'standard_scaler__Tenure', 'standard_scaler__Balance',
       'standard_scaler__NumOfProducts', 'standard_scaler__HasCrCard',
       'standard_scaler__IsActiveMember', 'standard_scaler__EstimatedSalary',
       'categorical_scaler__Geography_France',
       'categorical_scaler__Geography_Germany',
       'categorical_scaler__Geography_Spain',
       'categorical_scaler__Gender_Female', 'categorical_scaler__Gender_Male',
       'Exited'],
      dtype='object')

In [185]:
x_train=train_data.drop(["Exited"],axis=1)
y_train=train_data["Exited"]

In [186]:
x_test=test_data.drop(["Exited"],axis=1)
y_test=test_data["Exited"]

In [187]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((7500, 14), (7500,), (2500, 14), (2500,))

In [188]:
def get_model(model_input_shape):
  n=model_input_shape
  model=Sequential()
  model.add(
      Dense(
          units=2*n,
          activation='relu',
          input_shape=(n,)
      )
  )
  model.add(
      Dense(
          units=4*n,
          activation="relu"
      )
  )
  model.add(
      Dense(
          units=2*n,
          activation="relu"
      )
  )
  model.add(
      Dense(
          units=1,
          activation="sigmoid"
      )
  )

  print(model.summary())

  optimizer=Adam(learning_rate=0.005)
  loss=BinaryCrossentropy()
  metrics=BinaryAccuracy()

  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=[metrics]
  )
  return model

In [189]:
data_input_shape=x_train.shape[1]

In [190]:
model=get_model(data_input_shape)

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_32 (Dense)                │ (None, 28)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 56)             │         1,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 28)             │         1,596 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 1)              │            29 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,669 (14.33 KB)

 Trainable params: 3,669 (14.33 KB)

 Non-trainable params: 0 (0.00 B)

None


In [191]:
history=model.fit(
    x_train,
    y_train,
    validation_data=(x_test,y_test),
    epochs=100,
    batch_size=35,
    callbacks=[
        get_tensorboard_callback(),
        early_stopping_callback
    ]
)

Epoch 1/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.6793 - loss: 8.4375 - val_binary_accuracy: 0.3052 - val_loss: 0.9814
Epoch 2/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7151 - loss: 1.0061 - val_binary_accuracy: 0.8016 - val_loss: 0.4905
Epoch 3/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7515 - loss: 0.7275 - val_binary_accuracy: 0.8012 - val_loss: 0.4856
Epoch 4/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7699 - loss: 0.5912 - val_binary_accuracy: 0.8056 - val_loss: 0.4608
Epoch 5/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7884 - loss: 0.5060 - val_binary_accuracy: 0.8016 - val_loss: 0.4537
Epoch 6/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7628 - loss: 0.6216 - val_binary_accuracy: 0.8048 - val_loss: 0.4697
Epoch 7/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7909 - loss: 0.4857 - val_binary_accuracy: 0.8012 - val_loss: 0.4520

In [198]:
launch_tensorboard()

Launching TensorBoard at: e:\GitHubFIles\py_play_ground\dl\basic\log\fit\04_08_25__11_35_36


Reusing TensorBoard on port 6024 (pid 2608), started 0:00:29 ago. (Use '!kill 2608' to kill it.)

In [193]:
history=model.fit(
    x_train,
    y_train,
    validation_data=(x_test,y_test),
    epochs=100,
    batch_size=int(data_input_shape/25),
    callbacks=[
        get_tensorboard_callback(),
        early_stopping_callback
    ]
)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7776 - loss: 0.5983 - val_binary_accuracy: 0.8016 - val_loss: 0.5058
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7949 - loss: 0.5078 - val_binary_accuracy: 0.8012 - val_loss: 0.4994
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7949 - loss: 0.5064 - val_binary_accuracy: 0.8012 - val_loss: 0.4998
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7951 - loss: 0.5059 - val_binary_accuracy: 0.8012 - val_loss: 0.4998
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7952 - loss: 0.5053 - val_binary_accuracy: 0.8024 - val_loss: 0.4955
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7953 - loss: 0.5063 - val_binary_accuracy: 0.8008 - val_loss: 0.5022
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7953 - loss: 0.5055 - val_binary_accuracy: 0.8000 - val_loss: 0.5029

In [199]:
launch_tensorboard()

Launching TensorBoard at: e:\GitHubFIles\py_play_ground\dl\basic\log\fit\04_08_25__11_35_36


Reusing TensorBoard on port 6024 (pid 2608), started 0:00:40 ago. (Use '!kill 2608' to kill it.)

In [195]:
history=model.fit(
    x_train,
    y_train,
    validation_data=(x_test,y_test),
    epochs=100,
    batch_size=int(data_input_shape/25),
    callbacks=[
        get_tensorboard_callback(),
        early_stopping_callback
    ]
)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7948 - loss: 0.5068 - val_binary_accuracy: 0.8016 - val_loss: 0.5014
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7949 - loss: 0.5062 - val_binary_accuracy: 0.8016 - val_loss: 0.5012
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7949 - loss: 0.5060 - val_binary_accuracy: 0.8008 - val_loss: 0.5014
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7831 - loss: 0.9470 - val_binary_accuracy: 0.6676 - val_loss: 0.6111
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7920 - loss: 0.5690 - val_binary_accuracy: 0.8016 - val_loss: 0.5020
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7951 - loss: 0.5050 - val_binary_accuracy: 0.8008 - val_loss: 0.5032
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7951 - loss: 0.5078 - val_binary_accuracy: 0.8012 - val_loss: 0.5037

In [200]:
launch_tensorboard()

Launching TensorBoard at: e:\GitHubFIles\py_play_ground\dl\basic\log\fit\04_08_25__11_35_36


Reusing TensorBoard on port 6024 (pid 2608), started 0:00:50 ago. (Use '!kill 2608' to kill it.)

In [197]:
model_file_name="model.h5"
save_model(model,model_file_name)
